# 写入磁盘数据
一旦我们加载了数据，可能需要将其保存为某种磁盘格式。Satpy 提供了多种格式，可以通过 "writer" 对象将我们的数据保存为这些格式。让我们使用 Satpy 的实用函数来了解有哪些读取器可供我们使用：

In [ ]:
%run ../init_notebook.py
from satpy import available_writers
sorted(available_writers())

在上述输出中，你应该能看到 3-5 个写入器，这取决于你的 Python 环境配置。在本教程中，我们将使用 `geotiff`、`simple_image` 和 `cf` 写入器。如果某些写入器缺少依赖项，它们将不会在此处列出。我们可以通过使用 `check_satpy` 来检查缺少的功能，就像我们在 [简介](./01_introduction.ipynb) 中所做的那样。
## 图像 - GeoTIFF
我们将从学习如何保存geotiffs，使用Satpy的默认写入器，使用上一课中使用的相同数据开始。让我们创建一个`Scene`对象并选择16个ABI通道中的一个（例如`C07`）。

In [ ]:
from satpy import Scene
from glob import glob

my_channel = 'EDITME'
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.load([my_channel])

最简单的方法之一是使用 `Scene` 的 `save_datasets` 方法将所有加载的数据保存到磁盘。

In [ ]:
scn.save_datasets()

默认情况下，此方法会将文件保存到当前工作目录 (`.`)。
使用笔记本的 `!` 功能，我们可以运行命令行工具，并使用 GDAL 项目的 [gdalinfo](https://gdal.org/programs/gdalinfo.html) 来检查文件：

In [ ]:
!pwd
!ls

In [ ]:
!gdalinfo C??_20180511_213220.tif

Satpy 已创建了一个 LA（亮度 + 透明度）条带状 GeoTIFF 图像，使用了 "DEFLATE" 压缩算法进行压缩。从 `gdalinfo` 的输出中，我们可以看到诸如以 Well-known Text (WKT) 格式表示的地图投影参数、图像尺寸和图像数据类型等信息。GeoTIFF 不是一种标准的网络图像格式，因此我们无法在浏览器中直接显示它们。它们通常由 GIS 工具如 ArcGIS、QGIS 或网络地图服务 (WMS) 使用。大多数图像查看器也可以显示这些图像，因此导航到图像文件并打开它（双击）应该会使用 Linux 上的 ImageMagick、macOS 上的 预览 以及 Windows 上的 Windows 照片查看器 打开图像。
在幕后，Satpy 使用 `rasterio` 和 `GDAL` 库来创建 GeoTIFF。GDAL 为 GeoTIFF 创建提供了 [许多选项](https://gdal.org/drivers/raster/gtiff.html#creation-options)，而 Satpy 允许我们在保存时指定这些选项。让我们创建一个分块的 GeoTIFF 图像，这在将图像托管在云服务上时会很有用（详见 [COG](https://www.cogeo.org/)），并且通常会产生更小的文件。我们还将使用一个名为 `ProgressBar` 的 Dask 工具来提供计算过程的反馈。

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar():
    scn.save_datasets(tiled=True, copy_src_overviews=True)

Satpy 覆盖了我们之前生成的文件。我们再次使用 `gdalinfo` 来检查该文件。

In [ ]:
!ls

In [ ]:
!gdalinfo C??_20180511_213220.tif

主要区别在于 `Block=256x256`，这告诉我们数据是以 256x256 的块进行存储的；这是云优化GeoTIFF推荐的存储方式。
我们可以在上面的 `gdalinfo` 输出中看到，我们的图像数据类型为 `Byte`，这意味着它是一个 8 位无符号整数。有时你可能希望将数据以浮点数的形式存储在 GeoTIFF 中。让我们再做一个 GeoTIFF 示例，并使用一些额外的关键字参数来定制我们的输出。

In [ ]:
import numpy as np

with ProgressBar():
    scn.save_datasets(base_dir='float_images', filename='{name}.float.tif',
                      dtype=np.float32, enhance=False)

In [ ]:
!ls

In [ ]:
!gdalinfo float_images/C??.float.tif

我们现在创建了一个32位浮点型GeoTIFF文件，用于GIS应用或其他软件进行更深入的分析。我们使用了`base_dir`关键字参数来指定一个新目录，用于保存我们的文件。我们指定了`enhance=False`参数，以指示Satpy不要将观测数据值（开尔文温度、百分比等）缩放到图像值（0-255）；后续课程中将对此进行更详细的讲解。我们还通过`filename`关键字参数提供了一个自定义的文件命名方案。
### 练习
**时间: 5 分钟**
一个关键点是，文件命名方案可以使用 DataArray 的 `.attrs` 字典中的任何属性。要尝试这一点，请更新之前 `save_datasets` 调用中的文件名模式，添加 `standard_name` 或 `start_time` 或任何其他属性。生成的文件名是什么？

## 图片 - PNG、JPEG 等
到目前为止，我们使用 `'geotiff'` 写入器创建了 GeoTIFF。但通过使用 `'simple_image'` 写入器，我们也可以创建 PNG 和其他基本图像格式，该写入器利用了 PIL 库支持的多种格式。

In [ ]:
with ProgressBar():
    scn.save_datasets(writer='simple_image')

In [ ]:
!ls

我们可以像查看GeoTIFF图像一样，使用系统自带的图像查看器来查看这张图像，或者直接在浏览器中查看。然而，默认情况下，Satpy已将整个图像数组保存到文件中，因此查看其中几个可能会对浏览器造成问题。在未来的课程中，我们将学习在将数据保存到磁盘之前调整其分辨率的方法。
与其显式指定 ``writer``，我们可以让 Satpy 根据文件名确定最佳的写入器。根据我们之前对 ``filename`` 的了解，让我们更新下面的调用，以保存带有 ``.png`` 扩展名的 PNG 图像。

In [ ]:
scn.save_datasets(editme=editme)

# 符合CF规范的NetCDF文件
除了图像格式外，我们可能还希望将数据保存为更常见的数据文件格式。在处理卫星影像数据时，一种非常常见的格式是带有符合气候与预报（Climate and Forecast，CF）元数据约定的元数据的NetCDF格式（.nc）。我们将使用`'cf'`写入器，它是所有以`.nc`结尾的文件的默认写入器。
让我们创建一个新的场景，加载几个通道，并将其保存到NetCDF文件中。

In [ ]:
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_conus/*.nc')
scn = Scene(reader='abi_l1b', filenames=filenames)
scn.load(['C07', 'C08', 'C09'])

In [ ]:
with ProgressBar():
    scn.save_datasets(filename='my_abi_data.nc', engine='netcdf4')

In [ ]:
!ls

In [ ]:
!ncdump -h my_abi_data.nc

注意这个处理方式还为我们处理了x/y坐标变量以及CF的`grid_mapping`变量。默认情况下，Satpy还会为那些更习惯使用角度而非投影米的用户创建`longitude`（经度）和`latitude`（纬度）变量。对于这个写入器，我们有许多可配置的选项，包括关闭经度和纬度变量的创建、变量分组等，详见 [许多选项](https://satpy.readthedocs.io/en/latest/api/satpy.writers.html#module-satpy.writers.cf_writer)。探索这些选项超出了本教程的范围，留作读者练习。

### 进一步研究
* [Scene.save_dataset][1]* [GeoTIFF 创建选项][2]* [NetCDF 创建选项][3]
[1]: https://satpy.readthedocs.io/en/latest/api/satpy.html#satpy.scene.Scene.save_dataset[2]: https://gdal.org/drivers/raster/gtiff.html#creation-options[3]: https://satpy.readthedocs.io/en/latest/api/satpy.writers.html#module-satpy.writers.cf_writer